https://medium.com/the-modern-scientist/monitoring-machine-learning-models-an-in-depth-look-at-evidently-ai-dfebd5760643

In [3]:
try:
    import evidently
except:
    !pip install git+https://github.com/evidentlyai/evidently.git

In [1]:
!pip uninstall -y evidently


Found existing installation: evidently 0.6.5
Uninstalling evidently-0.6.5:
  Successfully uninstalled evidently-0.6.5


In [2]:
!pip install --upgrade evidently

  Using cached evidently-0.6.5-py3-none-any.whl.metadata (11 kB)
Using cached evidently-0.6.5-py3-none-any.whl (3.5 MB)


In [3]:
!pip show evidently

Name: evidently
Version: 0.6.5
Summary: Open-source tools to analyze, monitor, and debug machine learning model in production.
Home-page: https://github.com/evidentlyai/evidently
Author: Emeli Dral
Author-email: emeli.dral@gmail.com
License: UNKNOWN
Location: /opt/anaconda3/envs/torch/lib/python3.12/site-packages
Requires: certifi, cryptography, deprecation, dynaconf, fsspec, iterative-telemetry, litestar, nltk, numpy, pandas, plotly, pydantic, PyYAML, requests, rich, scikit-learn, scipy, statsmodels, typer, typing-inspect, ujson, urllib3, uuid6, uvicorn, watchdog
Required-by: 


In [4]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset, ClassificationPreset
from evidently.options import ColumnMapping  # Updated import

# Step 1: Create synthetic dataset
X, y = make_classification(n_samples=1000, n_features=20, n_informative=2, n_redundant=10, random_state=42)
data = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(X.shape[1])])
data['target'] = y

# Step 2: Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data.drop('target', axis=1), data['target'], test_size=0.25, random_state=42)

# Train a model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Predict on the test set
predictions = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
print(f'Model Accuracy: {accuracy}')

# Step 3: Prepare data for Evidently
reference_data = pd.concat([X_train, y_train], axis=1)
current_data = X_test.copy()
current_data['target'] = y_test
current_data['prediction'] = predictions

# Define column mapping using ColumnMapping
column_mapping = ColumnMapping(
    target='target',
    prediction='prediction',
    numerical_features=[f'feature_{i}' for i in range(20)],
    categorical_features=[]
)

# Generate report
report = Report(metrics=[DataDriftPreset(), ClassificationPreset()])
report.run(reference_data=reference_data, current_data=current_data, column_mapping=column_mapping)
report.save_html('evidently_report.html')

ImportError: cannot import name 'ColumnMapping' from 'evidently.options' (/opt/anaconda3/envs/torch/lib/python3.12/site-packages/evidently/options/__init__.py)